In [3]:
!pip install tiktoken


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [8]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

In [5]:
with open("input.txt", "r") as f:
    text = f.read()

In [7]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
encoded_text = encoding.encode(text)

In [11]:
len(encoded_text)

301829

In [12]:
len(text)

1115394

In [1]:
from src.dataset import TextData, NaiveEncoder

In [8]:
dataset = TextData("input.txt", batch_size=4, block_size=32)

In [9]:
x, y = dataset.get_batch()

In [10]:
x.shape

torch.Size([4, 32])

In [11]:
encoder = NaiveEncoder(text)

In [12]:
encoder.decode(x[0, :].numpy().tolist())

'uld think.\n\nSecond Servingman:\nS'

In [10]:
encoding.n_vocab

100277